#### readme:
整合来自近邻计数和KDE近邻求和的数据

合并为两个DataFrame:
1.kde_total
2.nei_total

最终输出：data_ori_z_05_35_sort_by_mpc_add_param.csv
整合了原有物理参数以及kde,nei数据

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import os
from scipy import stats
from astropy.coordinates import SkyCoord
import seaborn as seaborn
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import Planck15 as cosmo

#### 合并csv文档

In [2]:
r_kpc_=[20,40,60,80,100,200,400,500,800,1000,1200,1400,2000,2200,2600,3000]

kde_total=pd.read_csv('./output_csv/kde_sum_20.csv')
nei_total=pd.read_csv('./output_csv/neighbour_3D_seperation_20.csv')

for i in r_kpc_:
        kde_tem=pd.read_csv('./output_csv/kde_sum_{}.csv'.format(i))
        nei_tem=pd.read_csv('./output_csv/neighbour_3D_seperation_{}.csv'.format(i))

        kde_total['kde_sum_{}'.format(i)]=kde_tem['0']
        nei_total['nei_sum_{}'.format(i)]=nei_tem['0']

kde_total.drop(['Unnamed: 0','0'],axis=1,inplace=True)
nei_total.drop(['Unnamed: 0','0'],axis=1,inplace=True)

#### 得到每个尺寸下的平均kde

In [3]:
kde_average=pd.DataFrame(-999*np.ones((len(kde_total),len(r_kpc_))))

for j in range(len(r_kpc_)):
    for i in range(len(kde_total)):
        if nei_total.iloc[i,j]==0:
            kde_average.iloc[i,j]=kde_total.iloc[i,j]
        else:
            ttt=(kde_total.iloc[i,j])/(nei_total.iloc[i,j])
            kde_average.iloc[i,j]=ttt      # 得到每个尺寸下的平均kde

In [4]:
len(kde_average)

181320

#### 计算不同尺度下的 density

In [5]:
density=pd.DataFrame(999*np.ones((len(kde_total),len(r_kpc_))))
density.columns=['density_20','density_40','density_60','density_80','density_100','density_200','density_400','density_500','density_800','density_1000','density_1200','density_1400','density_2000','density_2200','density_2600','density_3000']

for n,c in enumerate(density.columns):
    tem=kde_average.iloc[:,n]/np.log(r_kpc_[n])
    density[c]=tem

In [6]:
len(density)

181320

#### 将星系物理信息和kde,nei数据合并

In [7]:
# 所有数据
data_ori_z_05_35=pd.read_csv('data_ori_z_05_35_sort_by_mpc.csv')


data_ori_z_05_35['ssfr']=data_ori_z_05_35['lp_SFR_med']-data_ori_z_05_35['lp_mass_best']

In [8]:
for c in nei_total.columns:
    data_ori_z_05_35[c]=nei_total[c]


for c in density.columns:
    data_ori_z_05_35[c]=density[c]

In [9]:
data_ori_z_05_35

,ID,RA,DEC,X_IMAGE,Y_IMAGE,HSC_g_FLUXERR_APER3,HSC_g_MAG_APER3,HSC_g_MAGERR_APER3,HSC_r_FLUXERR_APER3,HSC_r_MAG_APER3,...,density_400,density_500,density_800,density_1000,density_1200,density_1400,density_2000,density_2200,density_2600,density_3000
0,884529,150.031908,2.256563,24739.855469,23175.228516,0.033664,22.237005,0.008365,0.050496,21.317850,...,0.006634,0.006396,0.005946,0.005754,0.005606,0.005487,0.007824,0.007727,0.005790,0.005518
1,653172,149.830656,2.045145,29567.189453,18101.722656,0.013434,24.798761,0.035379,0.018970,23.779758,...,0.005525,0.005327,0.004953,0.004793,0.004669,0.004570,0.008853,0.008744,0.006423,0.005183
2,853993,149.993710,2.232933,25655.943359,22608.162109,0.008699,27.485142,0.271876,0.010708,27.026620,...,0.006597,0.006360,0.005913,0.005722,0.005575,0.005456,0.007677,0.007582,0.007420,0.006126
3,962886,150.021089,2.331814,24999.199219,24981.271484,0.009622,26.139133,0.086754,0.012907,25.240316,...,0.006306,0.006080,0.005652,0.005470,0.005329,0.005216,0.009997,0.009873,0.006166,0.005487
4,1020596,149.872515,2.384318,28561.837891,26241.798828,0.010247,26.153310,0.093768,0.012874,25.350235,...,0.005826,0.005617,0.005222,0.005053,0.004923,0.004819,0.004592,0.004536,0.006583,0.005412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181315,855227,150.174640,2.233648,21316.908203,22625.222656,0.008858,26.649893,0.127248,0.012080,25.620985,...,0.000470,0.000453,0.000422,0.000408,0.000397,0.000389,0.000371,0.000366,0.000358,0.000352
181316,813369,149.860418,2.196797,28852.714844,21741.304688,0.009337,26.933392,0.175967,0.011879,26.061490,...,0.000593,0.000571,0.000531,0.000514,0.000501,0.000490,0.000467,0.000461,0.000452,0.000443
181317,813342,149.821869,2.196782,29777.218750,21741.103516,0.008665,29.330772,1.484834,0.011200,27.079970,...,0.000565,0.000545,0.000507,0.000490,0.000478,0.000468,0.000446,0.000440,0.000431,0.000423
181318,909936,149.995590,2.284049,25610.750000,23834.945312,0.009132,27.180273,0.215387,0.011067,26.431720,...,0.000556,0.000536,0.000498,0.000482,0.000469,0.000459,0.000438,0.000432,0.000423,0.000416


In [10]:
# data_ori_z_05_35.to_csv('data_ori_z_05_35_sort_by_mpc_add_param__ssfr_corrected.csv',index=False)